In [79]:
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine, types

# Define um mapa de tipos entre os tipos nos layouts e no SQLAlchemy
MAPA_TIPOS = {
    'VARCHAR2': types.String,
    'CHAR': types.String,
    'NUMBER': types.Numeric,
    'DATE': types.Date
}
# Define o caminho absoltuo do sigtap
CAMINHO_SIGTAP = Path('sigtap-simplificado')

# Retorna um dicionário contendo DataFrames com os conteúdos de cada layout
def gera_df_layouts(sigtap_dir: Path = CAMINHO_SIGTAP) -> dict:

    return {
        arquivo.stem.replace('_layout', ''): pd.read_csv(arquivo) # Como o conteúdo do txt de layout está estruturado em csv, lê arquivo dessa forma diretamente
        for arquivo in sigtap_dir.glob('*layout.txt') # Faz isso para cada arquivo de layout
    }

# Retorna um dict contendo DataFrames com os conteúdo de cada tabela ou relacionamento
def gera_df_tabelas(sigtap_dir: Path = CAMINHO_SIGTAP) -> dict:

    # Define dict que armazenará conteúdos das tabelas e relacionamentos
    dict_tb_rl = {}
    # Define dict de layouts, que será necessário para a criação dos DataFrames de tabelas e relacionamentos
    dict_layouts = gera_df_layouts(sigtap_dir)

    # Para cada arquivo de tabela ou relacionamento
    for arquivo in sigtap_dir.glob('*.txt'):
        if arquivo.stem.endswith('_layout'): # Ignora caso seja layout
            continue
        
        # Define variáveis para auxiliar a leitura dos conteúdos em texto
        nome_tb_rl = arquivo.stem
        df_layout = dict_layouts[nome_tb_rl] # Define layout da tabela atual

        # Define as especificações para leitura do conteúdo
        colspecs = list(zip(df_layout['Inicio'] - 1, df_layout['Fim']))

        # Cria DataFrame utilizando read_fwf, que aplica a especificação ao conteúdo do arquivo
        df = pd.read_fwf(
            arquivo,
            colspecs=colspecs,
            names=df_layout['Coluna'].tolist(),
            dtype=str,
            header=None,
            encoding='latin-1'
        )
        # Remove os espaços vazios nas colunas
        df = df.apply(lambda col: col.str.strip())
        dict_tb_rl[nome_tb_rl] = df

    return dict_tb_rl

# Define função que resolve o tipo recebido do layout
def resolve_tipo(tipo_layout: str, tamanho: int) -> types:

    # Extrai tipo do mapa de tipos. Caso não haja correspondência, define Text por padrão
    tipo = MAPA_TIPOS.get(tipo_layout, types.Text)

    # Retorna o tipo com o tamanho definido pelo layout
    if tipo is types.String:
        return tipo(tamanho)
    
    return tipo

# Cria tabelas no banco de dados a partir dos DataFrames
def cria_tabelas(dict_tb_rl: dict, dict_layouts: dict, caminho_bd: str) -> None:

    engine = create_engine(f'sqlite:///{caminho_bd}')

    with engine.begin() as conn:
        for nome_tb_rl, df in dict_tb_rl.items():
            layout = dict_layouts[nome_tb_rl]

            dtype = {
                row.Coluna: resolve_tipo(row.Tipo, int(row.Tamanho))
                for row in layout.itertuples()
            }

            df.to_sql(
                name=nome_tb_rl,
                con=conn,
                if_exists='replace',
                index=False,
                dtype=dtype
            )

            print(f'Tabela {nome_tb_rl} criada com {len(df)} registros.')

In [80]:
cria_tabelas(gera_df_tabelas(), gera_df_layouts(), 'sigtap.bd')

Tabela rl_procedimento_cid criada com 80610 registros.
Tabela tb_cid criada com 14238 registros.
Tabela tb_forma_organizacao criada com 395 registros.
Tabela tb_grupo criada com 8 registros.
Tabela tb_procedimento criada com 4730 registros.
Tabela tb_sub_grupo criada com 59 registros.
